# Deck Z₂ symmetry reduction (C24 clock) — full 59×24 Hamiltonian

This notebook demonstrates the **deck Z₂** symmetry (phase → phase+3 inside each sector) and uses it to **block-diagonalize** the full orbit-0 Hamiltonian on the 59×24 state space.

It is designed to run in the repository environment described in `ENV.md` (Python 3.11). If you also have Sage/PySymmetry installed, an optional section shows how to reproduce the same split via PySymmetry’s API.


## 0) Setup
We will read the canonical sparse matrices under `data/_toe/projector_recon_20260110/`:
- `N12_58_orbit0_H_transport_59x24_sparse_...npz`
- `N12_58_sector_coin_C24_K4_by_k_sparse_...npz`
- `TOE_H_total_transport_plus_lambda_coin_59x24_lam0.5_...npz`

The repo’s pinned Python deps are in `requirements.txt`, and Sage/PySymmetry is optional per `ENV.md`.


In [ ]:

from __future__ import annotations
import math
from pathlib import Path

import numpy as np
import scipy.sparse as sp

ROOT = Path("..").resolve()   # notebook in notebooks/, so .. is repo root
DATA = ROOT / "data"

PROJ_RECON = DATA / "_toe" / "projector_recon_20260110"
H_TRANSPORT_NPZ = PROJ_RECON / "N12_58_orbit0_H_transport_59x24_sparse_20260109T205353Z.npz"
COIN_NPZ = PROJ_RECON / "N12_58_sector_coin_C24_K4_by_k_sparse_20260109T205353Z.npz"
H_TOTAL_NPZ = PROJ_RECON / "TOE_H_total_transport_plus_lambda_coin_59x24_lam0.5_20260109T205353Z.npz"
EDGES_CSV = PROJ_RECON / "N12_58_orbit0_edges_with_2T_connection_20260109T043900Z.csv"

assert H_TRANSPORT_NPZ.exists()
assert COIN_NPZ.exists()
assert H_TOTAL_NPZ.exists()
assert EDGES_CSV.exists()

def load_csr_npz(path: Path) -> sp.csr_matrix:
    z = np.load(path, allow_pickle=True)
    return sp.csr_matrix((z["data"], z["indices"], z["indptr"]), shape=tuple(z["shape"]))

H_transport = load_csr_npz(H_TRANSPORT_NPZ)
coin = load_csr_npz(COIN_NPZ)
H_total = load_csr_npz(H_TOTAL_NPZ)

H_transport.shape, coin.shape, H_total.shape, H_total.nnz


## 1) Define the deck swap Z₍₂₄₎
Deck swap acts by **phase → phase+3 (mod 6)** inside each of 4 sectors. This is the same `e*` action in the 2T clock alignment.


In [ ]:

def deck_swap_24() -> sp.csr_matrix:
    rows, cols, data = [], [], []
    for sector in range(4):
        base = sector * 6
        for phase in range(6):
            i = base + phase
            j = base + ((phase + 3) % 6)
            rows.append(i); cols.append(j); data.append(1.0)
    return sp.csr_matrix((data, (rows, cols)), shape=(24, 24))

Z24 = deck_swap_24()

# Full Z on 59×24 space
I59 = sp.eye(59, format="csr")
Z = sp.kron(I59, Z24, format="csr")

# Commutator norms (should be exactly 0 in this dataset)
comm_coin = (Z24 @ coin) - (coin @ Z24)
comm_H = (Z @ H_total) - (H_total @ Z)

float(np.linalg.norm(comm_coin.data)), float(np.linalg.norm(comm_H.data))


## 2) Build the explicit symmetry-adapted basis (even/odd)
For each swapped pair (i, j=i+3), define:
- even basis vector: (eᵢ + eⱼ)/√2
- odd basis vector:  (eᵢ − eⱼ)/√2

This yields 24 = 12 ⊕ 12, and on the full space 1416 = 708 ⊕ 708.


In [ ]:

def deck_pairs():
    pairs=[]
    for s in range(4):
        base = s*6
        for ph in range(3):
            pairs.append((base+ph, base+ph+3))
    return pairs

pairs = deck_pairs()
len(pairs), pairs[:6]


In [ ]:

def build_T_even_odd_from_pairs(pairs):
    n_pairs=len(pairs)
    val=1.0/math.sqrt(2.0)
    rows_e=[]; cols_e=[]; data_e=[]
    rows_o=[]; cols_o=[]; data_o=[]
    for k,(i,j) in enumerate(pairs):
        rows_e += [i,j]; cols_e += [k,k]; data_e += [val,val]
        rows_o += [i,j]; cols_o += [k,k]; data_o += [val,-val]
    Te = sp.csr_matrix((data_e,(rows_e,cols_e)), shape=(24,n_pairs))
    To = sp.csr_matrix((data_o,(rows_o,cols_o)), shape=(24,n_pairs))
    return Te, To

Te, To = build_T_even_odd_from_pairs(pairs)

Ke = sp.kron(I59, Te, format="csr")  # 1416×708
Ko = sp.kron(I59, To, format="csr")  # 1416×708

Ke.shape, Ko.shape, (Ke.T @ Ke).shape


### Block-diagonalize H_total
Compute:
- H_even = Keᵀ H Ke
- H_odd  = Koᵀ H Ko
- H_off  = Keᵀ H Ko

If H commutes with Z, then H_off must be exactly 0.


In [ ]:

H_even = Ke.T @ H_total @ Ke
H_odd  = Ko.T @ H_total @ Ko
H_off  = Ke.T @ H_total @ Ko

float(np.linalg.norm(H_off.data)), H_even.shape, H_odd.shape, H_even.nnz, H_odd.nnz


## 3) Build the **no-flux** transport by toggling the 4 defect edges
Your flux toggle is exactly: on the four δ4 edges, replace the edge element **e*** (deck swap) by **e** (identity).

Since the transport matrix is block-permutation on those edges, we can build H_transport(noflux) by replacing those 24×24 blocks with I₍₂₄₎.


In [ ]:

import pandas as pd

edges = pd.read_csv(EDGES_CSV)
defect = edges.loc[edges["edge_elem_2T"]=="e*", ["u","v"]]
defect_edges = [tuple(map(int,x)) for x in defect.to_numpy()]

defect_edges


In [ ]:

def set_block_lil(M_lil: sp.lil_matrix, u: int, v: int, block_mat: np.ndarray, d: int = 24):
    r0=u*d; c0=v*d
    for i in range(d):
        row = r0+i
        cols = M_lil.rows[row]
        data = M_lil.data[row]
        new_cols=[]; new_data=[]
        for col,val in zip(cols,data):
            if not (c0 <= col < c0+d):
                new_cols.append(col); new_data.append(val)
        M_lil.rows[row]=new_cols
        M_lil.data[row]=new_data
        for j in range(d):
            val = block_mat[i,j]
            if val != 0:
                M_lil[row, c0+j] = val

I24 = np.eye(24)

H_nf = H_transport.tolil(copy=True)
for u,v in defect_edges:
    set_block_lil(H_nf, u, v, I24)
    set_block_lil(H_nf, v, u, I24)

H_transport_noflux = H_nf.tocsr()

# sanity: each defect block is now identity
def block_dense(H, u, v, d=24):
    return H[u*d:(u+1)*d, v*d:(v+1)*d].toarray()
all(np.allclose(block_dense(H_transport_noflux,u,v), I24) for u,v in defect_edges), H_transport_noflux.nnz


### Compare flux vs no-flux in the Z₂ blocks
Let H_total(noflux) = H_transport(noflux) + 0.5·(I⊗coin).

The key claim (now *provable* from the dataset) is:
- ΔH_even = 0
- ΔH_odd  ≠ 0

i.e., flux insertion is a **pure deck-odd excitation**.


In [ ]:

H_total_noflux = H_transport_noflux + 0.5*sp.kron(I59, coin, format="csr")

H_nf_even = Ke.T @ H_total_noflux @ Ke
H_nf_odd  = Ko.T @ H_total_noflux @ Ko

d_even = H_even - H_nf_even
d_odd  = H_odd - H_nf_odd

float(np.linalg.norm(d_even.data)), float(np.linalg.norm(d_odd.data))


## 4) Optional: reproduce the same 24→12+12 split using PySymmetry (Sage)
Per `ENV.md`, Sage/PySymmetry is optional and best installed in a separate environment. fileciteturn4file0L28-L32  
If you are running inside Sage and have PySymmetry installed, you can:
- define G = C₂ with a single generator,
- define its representation via the permutation matrix Z24,
- call `quick_block_prevision` and `is_equivariant`.

PySymmetry exposes these methods explicitly. fileciteturn4file5L10-L23  
The expensive basis construction can be amortized over parameter sweeps. fileciteturn4file1L18-L25


In [ ]:

# OPTIONAL (requires Sage + PySymmetry). Safe to skip in plain Python.
try:
    from sage.all import CC, CyclicPermutationGroup, matrix  # type: ignore
    from pysymmetry import FiniteGroup, representation  # type: ignore

    def csr_to_sage(m: sp.csr_matrix):
        entries = {}
        indptr = m.indptr; indices=m.indices; data=m.data
        for i in range(m.shape[0]):
            for k in range(indptr[i], indptr[i+1]):
                j = int(indices[k]); v = data[k]
                if v != 0:
                    entries[(i,j)] = CC(v)
        return matrix(CC, m.shape[0], m.shape[1], entries, sparse=True)

    Z24_sage = csr_to_sage(Z24)
    coin_sage = csr_to_sage(coin)

    G = FiniteGroup(CyclicPermutationGroup(2), field=CC)
    gens = G.gens()
    rep = representation(gens, [Z24_sage], field=CC)

    block_info = G.quick_block_prevision(rep, block_prevision=True)
    equiv_coin = rep.is_equivariant_to(coin_sage)

    block_info, equiv_coin
except Exception as e:
    print("Skipping PySymmetry section:", repr(e))


## 5) Export blocks for faster downstream solves
You can now work with 708×708 blocks rather than the full 1416×1416 operator.


In [ ]:

OUT = DATA / "_workbench" / "05_symmetry"
OUT.mkdir(parents=True, exist_ok=True)

sp.save_npz(OUT / "H_total_flux_even_708.npz", H_even.tocsr())
sp.save_npz(OUT / "H_total_flux_odd_708.npz", H_odd.tocsr())
sp.save_npz(OUT / "H_total_noflux_even_708.npz", H_nf_even.tocsr())
sp.save_npz(OUT / "H_total_noflux_odd_708.npz", H_nf_odd.tocsr())

print("Wrote blocks to", OUT)
